### Quasiquote (backquote) `` ` ``, unquote `,` e unquote-splicing `,@`

Convenhamos que ficar escrevendo `(list` junto a `'simb` para simbolos que não queremos computar e somente `simb` para simbolos que queremos computar não é muito elegante.

Por conta disso, scheme vem equipado com um reader macro chamado de **quasiquote** ou **backquote** (`` ` ``), ele funciona muito parecido com o **quote** (`'`):

In [1]:
`(x y 1 2)

(x y 1 2)

Entretando, quando combinado com outro reader macros **unquote** (`,`) ele é capaz de computar valores sob demanda, por exemplo:

In [1]:
(define x 1)
(define y 2)

`((x ,x) (y ,y))

((x 1) (y 2))

### Questão

O que o código a seguir retorna? 

```scheme
(define x 'y)
`,x 
```

### Questão

E o código a seguir?
```scheme
(define x 'y)
`',x
```

Além disso, temos também o reader macro **unquote-splicing** (`,@`) que é meio complicado de entender, mas em poucas palavras ele "remove 1 par de parenteses de uma lista".

Ou seja:


In [9]:
(define x '(1 2 3 4))

`(0 ,@x)

(0 1 2 3 4)

Viram como a lista `'(1 2 3 4)` se tornou `1 2 3 4`?

O **unquote-splicing** é poderosissimo e utilizado para trabalhar com macros que recebem um número váriavel de argumentos, por exemplo:

In [2]:
(define (f . args)
  `(begin ,@args))

(f '(display "oi")
   '(newline) 
   '(display "tchau") 
   '(newline))

(begin (display "oi") (newline) (display "tchau") (newline))

Ou seja, em tempo de compilação, nos tinhamos uma lista de listas: 

```scheme
'((display "oi")
  (newline)
  (display "tchau")
  (newline))
```

Que foi transformado em um código:

```scheme
(begin (display "oi") 
       (newline) 
       (display "tchau")
       (newline))
```

Vamos mostrar o comportamento com um macro:

In [4]:
(define-macro (m . args)
  `(begin ,@args))

(m (display "oi")
   (newline) 
   (display "tchau") 
   (newline))

oi
tchau


Em resumo, utilizamos **unquote-splicing** para adicionar elementos no inicio ou no fim de uma série de instruções sem precisar utilizar funções que manipulam a lista.

#### Questão
Crie um macro `desafio`, utilizando **somente** **quasiquote**, **unquote** e **unquote-splicing** que recebe um argumento `arg1`, uma função `f`, e um número váriavel de argumentos `args` e retorna uma aplicação de `f` a `args` e `arg1`.

Exemplo:

```scheme
(desafio 2 / 1 3 5) 

;; expande para

(/ 1 3 5 2)

;; computa para

1/30
```

In [1]:
(define-macro (desafio arg1 f . args)
  `(? ?? ???))

(desafio 2 / 1 3 5)

1/30

# PLANEJAMENTO
Implementar explicar o named-let (nlet):
```scheme
 (defmacro nlet (n letargs . body)
   `(letrec ([,n (lambda ,(map car letargs)
  		    ,@body)])
      (,n ,@(map cadr letargs))))
```

Pedir para implementarem o named-lambda (nlambda):
```scheme
(defmacro nlambda (n+args . body) 
  (let ([name (car n+args)]       
        [args (cdr n+args)])      
    `(letrec ([,name (lambda ,args
                       ,@body)])  
       ,name)))
```

Falar de váriaveis livres
Falar do problema da captura
Apresentar o gensym (como forçar higiene)
Falar do problema de higiene

Lab

Próxima aula -- Macros Higiênicos